In [2]:
from craterdetection.matching.database import CraterDatabase

In [5]:
%%time
db = CraterDatabase.from_file("../data/lunar_crater_database_robbins_2018.csv")

Wall time: 1min 18s


In [4]:
db.I_ij()

array([ -737.25724483, -1004.36682902,  -138.21755815,   -64.98201529,
       -1480.02170581,  -517.7849468 , -3259.11356447])